# 模型

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="https://api.lingyiwanwu.com/v1", 
    openai_api_key="multimodel-peter", 
    model="gpt-4o"
    )

print(llm.invoke("hi, who are you?"))


content="Hello! I'm an AI language model created by OpenAI, designed to help answer questions and provide information on a wide range of topics. How can I assist you today?" response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 13, 'total_tokens': 47}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6c38f160-771c-4503-805b-e4d13b1a466d-0' usage_metadata={'input_tokens': 13, 'output_tokens': 34, 'total_tokens': 47}


In [3]:
model = llm

In [4]:
from langchain_core.messages import HumanMessage
import base64

import httpx

# image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

image_url = "https://test-content-public.tos-cn-shanghai.volces.com/agent/others/%E5%B7%B4%E5%8E%98%E5%B2%9B3.jpeg"

image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

message = HumanMessage(
    content=[
        {"type": "text", "text": "描述图片中的天气"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = model.invoke([message])
print(response.content)



图片中的天气看起来晴朗，天空可能是蓝色的，阳光充足。海水呈现出明亮的蓝色和绿色，海浪拍打着沙滩，海滩上没有明显的阴影，显示出阳光直接照射。整体感觉是一个适合户外活动的好天气。


In [41]:
message = HumanMessage(content=[
        {"type": "text", "text": "计算这个数学公式，保留4位小数。"},
        # {"type": "text", "text": "计算这个数学公式，保留4位小数。请一步一步推理，有些步骤可以调用工具，从而确保最终答案正确。"},
        {
            "type": "image_url",
            "image_url": {"url": 'https://test-content-public.tos-cn-shanghai.volces.com/agent/others/complex_cal6.png'}
        },
    ])
response = model.invoke([message])
print(response.content)

首先我们计算分子的值：

\[ 
10 + 543^7 + 647 + 89854 + 12 
\]

其中，\[
543^7
\] 是一个非常大的数。让我们逐步计算：

\[
543^7 = 543 \times 543 \times 543 \times 543 \times 543 \times 543 \times 543
\]

通过计算我们得出：

\[
543^7 = 135,351,079,478,643,200,000
\]

加上其他数值：

\[
10 + 135,351,079,478,643,200,000 + 647 + 89854 + 12 \approx 135,351,079,478,643,299,523
\]

然后我们计算分母的值：

\[ 
8908 \times 0.495653324 \times 9 \times 143 \times 12 
\]

逐步计算：

\[
8908 \times 0.495653324 \approx 4415.5441
\]
\[
4415.5441 \times 9 \approx 39739.8969
\]
\[
39739.8969 \times 143 \approx 5682764.5667
\]
\[
5682764.5667 \times 12 \approx 68193174.8004
\]

然后，我们将分子的值除以分母的值：

\[ 
\frac{135,351,079,478,643,299,523}{68193174.8004} \approx 1.9849 \times 10^{21}
\]

保留4位小数，最终答案是：

\[ 
1.9849 \times 10^{21}
\]


In [5]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "描述图片中的天气"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

图片中显示的是一个海滩景观，天气看起来非常晴朗。蓝天白云，海水清澈见底，呈现出不同深浅的蓝色。海浪轻轻拍打着沙滩，沙滩洁白细腻。周围的岩石和植被显示出一片郁郁葱葱的景象，整体给人一种宁静和美丽的感觉。这样的天气适合海边活动，阳光明媚，温暖宜人。


In [6]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "这两张图是一个吗？"},
        {"type": "image_url", "image_url": {"url": image_url}},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

是的，这两张图是相同的。两张图显示的是同一个海滩和周边的景色，从角度、景物和颜色等各方面来看完全一致。


# tools

https://python.langchain.com/v0.2/docs/how_to/custom_tools/

In [7]:
from typing import Literal

from langchain_core.tools import tool


@tool
def weather_tool(weather: Literal["sunny", "cloudy", "rainy"]) -> None:
    """Describe the weather"""
    pass


model_with_tools = model.bind_tools([weather_tool])

message = HumanMessage(
    content=[
        {"type": "text", "text": "描述图片中的天气"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)

[{'name': 'weather_tool', 'args': {'weather': 'sunny'}, 'id': 'call_Npkm9n9omcf9uexay8lDUF59', 'type': 'tool_call'}]


## 创建同步tool

In [8]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [9]:
# import requests
# import time
# import os
# import json

# BASE_URI = 'https://api.bing.microsoft.com/v7.0/images/visualsearch'

# SUBSCRIPTION_KEY = 'ab817fc05fb84426a4df844b62b86eae'


# @tool
# def bing_visual_search(imagePath: str):
#     """利用bing搜索接口搜索图片路径对应的图片
#     """
#     resized_path= imagePath.split('.')[0]+f'_{int(time.time())}_resized.'+imagePath.split('.')[1]
#     resized_path = '/ML-A100/team/mm/alban/data/Yi_api_raw/bing_results/' + resized_path.split('/')[-1]
#     resize_image_to_under_1mb(imagePath, resized_path)
#     HEADERS = {'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY}
#     file = {'image' : ('myfile', open(resized_path, 'rb'))}
#     params  = {"count": 3}
#     try:
#         print('*****************bing visual searching***************')
#         response = requests.post(BASE_URI, headers=HEADERS, files=file, params=params, timeout=8)
#         response.raise_for_status()
#         search_results = response.json()
#         with open(resized_path.split('.')[0]+ ".json", 'w', encoding='utf-8') as f:
#             json.dump(search_results, f, ensure_ascii=False, indent=4)
#             #os.remove(resized_path)
#         related_prompts = extract_names(search_results)
#         print('*****************searched_result****************', related_prompts )

#         return related_prompts
#     except Exception as ex:
#         print(ex)
#         #raise ex
#         return ''


# def resize_image_to_under_1mb(input_path, output_path, initial_quality=85, size_decrement=0.9):
#     """
#     Resize an image so that its size is under 1MB by adjusting both quality and dimensions.

#     :param input_path: Path to the input image.
#     :param output_path: Path to save the resized image.
#     :param initial_quality: Initial quality of the image. Default is 85.
#     :param size_decrement: Factor by which to decrease the size. Default is 0.9 (i.e., 10% reduction each time).
#     :return: None
#     """
#     def save_image_with_quality(img, path, quality):
#         img.save(path, quality=quality)
#         return os.path.getsize(path)
    
#     with Image.open(input_path) as img:
#         if img.mode == 'RGBA':
#             img = img.convert('RGB')
        
#         # Initial save to check size
#         file_size = save_image_with_quality(img, output_path, initial_quality)
        
#         # Adjust quality first using binary search
#         low_quality, high_quality = 50, initial_quality
#         while file_size > 1 * 1024 * 1024 and high_quality > low_quality:
#             quality = (low_quality + high_quality) // 2
#             file_size = save_image_with_quality(img, output_path, quality)
#             if file_size > 1 * 1024 * 1024:
#                 high_quality = quality - 1
#             else:
#                 low_quality = quality + 1
        
#         # If quality adjustment is not enough, start resizing
#         while file_size > 1 * 1024 * 1024:
#             new_size = (int(img.size[0] * size_decrement), int(img.size[1] * size_decrement))
#             img = img.resize(new_size, Image.LANCZOS)
#             file_size = save_image_with_quality(img, output_path, high_quality)
            
#             if new_size[0] * new_size[1] < 64:
#                 raise ValueError("Cannot reduce image size to under 1MB without making it too small.")
        
#         # print(f"Final image size: {os.path.getsize(output_path)} bytes with quality: {high_quality} and dimensions: {img.size}")


# # 递归提取所有的 name 和 displayName 字段
# def extract_names(data, names=None, display_names=None):
#     if names is None:
#         names = []
#     if display_names is None:
#         display_names = []

#     if isinstance(data, dict):
#         for key, value in data.items():
#             if key == 'name' and value:
#                 names.append(value)
#             if key == 'displayName' and value:
#                 display_names.append(value)
#             extract_names(value, names, display_names)
#     elif isinstance(data, list):
#         for item in data:
#             extract_names(item, names, display_names)
#     # names_str = ', '.join(names[:10])
#     # display_names_str = ', '.join(display_names)
#     recall_str = '\n'.join([f'"{n}"' for n in names[:10] + display_names]) + '\n'
#     # return names_str + display_names_str +'\n'
#     return recall_str


In [10]:
import requests
import io
from PIL import Image
import json
import time

BASE_URI = 'https://api.bing.microsoft.com/v7.0/images/visualsearch'
SUBSCRIPTION_KEY = 'ab817fc05fb84426a4df844b62b86eae'

@tool
def bing_visual_search(imageUrl: str):
    """利用bing搜索接口搜索给定图片链接对应的图片
    """
    HEADERS = {'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY}
    params  = {"count": 3}

    try:
        print('*****************bing visual searching***************')
        # Download the image from the URL
        image_response = requests.get(imageUrl)
        image_response.raise_for_status()

        # Load the image into memory
        image_data = Image.open(io.BytesIO(image_response.content))

        # Resize the image to make sure its size is under 1MB
        resized_image = resize_image_to_under_1mb(image_data)

        # Convert the resized image back to bytes
        image_bytes = io.BytesIO()
        resized_image.save(image_bytes, format='JPEG')
        image_bytes.seek(0)

        # Use the resized image content directly
        file = {'image' : ('myfile', image_bytes)}

        response = requests.post(BASE_URI, headers=HEADERS, files=file, params=params, timeout=8)
        response.raise_for_status()
        search_results = response.json()

        # Save the search results to a JSON file
        json_filename = f'image_search_results_{int(time.time())}.json'
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(search_results, f, ensure_ascii=False, indent=4)

        related_prompts = extract_names(search_results)
        print('*****************searched_result****************', related_prompts)

        return related_prompts
    except Exception as ex:
        print(ex)
        return ''

def resize_image_to_under_1mb(image, initial_quality=85, size_decrement=0.9):
    """
    Resize an image so that its size is under 1MB by adjusting both quality and dimensions.

    :param image: PIL Image object.
    :param initial_quality: Initial quality of the image. Default is 85.
    :param size_decrement: Factor by which to decrease the size. Default is 0.9 (i.e., 10% reduction each time).
    :return: Resized PIL Image object.
    """
    def save_image_with_quality(img, quality):
        img_bytes = io.BytesIO()
        img.save(img_bytes, format='JPEG', quality=quality)
        img_bytes.seek(0)
        return img_bytes, len(img_bytes.getvalue())

    if image.mode == 'RGBA':
        image = image.convert('RGB')

    # Initial save to check size
    file_size = save_image_with_quality(image, initial_quality)[1]

    # Adjust quality first using binary search
    low_quality, high_quality = 50, initial_quality
    while file_size > 1 * 1024 * 1024 and high_quality > low_quality:
        quality = (low_quality + high_quality) // 2
        _, file_size = save_image_with_quality(image, quality)
        if file_size > 1 * 1024 * 1024:
            high_quality = quality - 1
        else:
            low_quality = quality + 1

    # If quality adjustment is not enough, start resizing
    while file_size > 1 * 1024 * 1024:
        new_size = (int(image.size[0] * size_decrement), int(image.size[1] * size_decrement))
        image = image.resize(new_size, Image.LANCZOS)
        _, file_size = save_image_with_quality(image, high_quality)

        if new_size[0] * new_size[1] < 64:
            raise ValueError("Cannot reduce image size to under 1MB without making it too small.")

    return image

# 递归提取所有的 name 和 displayName 字段
def extract_names(data, names=None, display_names=None):
    if names is None:
        names = []
    if display_names is None:
        display_names = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == 'name' and value:
                names.append(value)
            if key == 'displayName' and value:
                display_names.append(value)
            extract_names(value, names, display_names)
    elif isinstance(data, list):
        for item in data:
            extract_names(item, names, display_names)
    # names_str = ', '.join(names[:10])
    # display_names_str = ', '.join(display_names)
    recall_str = '\n'.join([f'"{n}"' for n in names[:10] + display_names]) + '\n'
    # return names_str + display_names_str +'\n'
    return recall_str

In [11]:
recall_str = bing_visual_search(image_url)

/opt/anaconda3/envs/agent/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


*****************bing visual searching***************
*****************searched_result**************** "巴厘岛-梦开始的地方.在异国他乡遇见你所有的美好.-巴厘岛旅游攻略-游记-去哪儿攻略"
"【乌鲁瓦图情人崖旅游】乌鲁瓦图情人崖旅游攻略，乌鲁瓦图情人崖旅游景点大全-去哪儿网"
"Perfect 3-day itinerary in Nusa Penida, Indonesia | Feast of Travel"
"Indonesia ~ Martina Move"
"7 Nights in Bali - Rewards4earth.org"
"1-Day Guide to Nusa Penida Island, Indonesia - Let's Travel it Up"
"Kelingkling beach from above | Etsy"
"Bali Travel Guide & Tips | Condé Nast Traveler"
"Bali Blue Dream Beach Background, Bali, Lembongan, Beach Background ..."
"West & East Nusa Penida Tour"
"Klumbu Indonesia"



In [24]:
# Define the tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

## 创建异步tool

In [12]:
@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

## 汇总tools

In [25]:
tools = [bing_visual_search, multiply, weather_tool, search]

## 调用tools

In [26]:
model_with_tools = model.bind_tools(tools)
model_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10eb2e170>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10eb2fa90>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_api_base='https://api.lingyiwanwu.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'bing_visual_search', 'description': '利用bing搜索接口搜索给定图片链接对应的图片', 'parameters': {'type': 'object', 'properties': {'imageUrl': {'type': 'string'}}, 'required': ['imageUrl']}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply two numbers.', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}}, {'type': 'function', 'function': {'name': 'weather_tool', 'description': 'Describe the weather', 'parameters': {'type': 'object', 'properties': {'weather': {'enum': ['sunny', 'cloudy', 'rainy'], 'type': 'string'}}, 

In [ ]:

message = HumanMessage(
    content=[
        {"type": "text", "text": "图片中的地方在哪个国家"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model_with_tools.invoke([message])
print(response.tool_calls)

In [15]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vexbnyMLXFmR1QQZFJcPcTlx', 'function': {'arguments': '{"imageUrl":"https://cdn.pixabay.com/photo/2018/05/17/21/39/kelingking-beach-3412359_1280.jpg"}', 'name': 'bing_visual_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 1019, 'total_tokens': 1066}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-881131d5-9b87-4106-be4b-feceb1c18da6-0', tool_calls=[{'name': 'bing_visual_search', 'args': {'imageUrl': 'https://cdn.pixabay.com/photo/2018/05/17/21/39/kelingking-beach-3412359_1280.jpg'}, 'id': 'call_vexbnyMLXFmR1QQZFJcPcTlx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1019, 'output_tokens': 47, 'total_tokens': 1066})

In [16]:
# 解析 tool_calls
tool_calls = response.additional_kwargs.get('tool_calls', [])
print(tool_calls)


[{'id': 'call_vexbnyMLXFmR1QQZFJcPcTlx', 'function': {'arguments': '{"imageUrl":"https://cdn.pixabay.com/photo/2018/05/17/21/39/kelingking-beach-3412359_1280.jpg"}', 'name': 'bing_visual_search'}, 'type': 'function'}]


In [21]:
# 从模型的响应中提取 tool_calls
tool_calls = response.tool_calls

# 迭代并处理每个工具调用
for tool_call in tool_calls:
    if tool_call['name'] == 'bing_visual_search':
        # 提取工具调用的参数
        args = tool_call['args']
        # image_url = args['imageUrl']
        imageUrl = image_url
        # 手动调用工具
        search_results = bing_visual_search(image_url)
        print(search_results)


*****************bing visual searching***************
*****************searched_result**************** "巴厘岛-梦开始的地方.在异国他乡遇见你所有的美好.-巴厘岛旅游攻略-游记-去哪儿攻略"
"【乌鲁瓦图情人崖旅游】乌鲁瓦图情人崖旅游攻略，乌鲁瓦图情人崖旅游景点大全-去哪儿网"
"巴厘岛-梦开始的地方.在异国他乡遇见你所有的美好.-巴厘岛旅游攻略-游记-去哪儿攻略"
"Perfect 3-day itinerary in Nusa Penida, Indonesia | Feast of Travel"
"Indonesia ~ Martina Move"
"7 Nights in Bali - Rewards4earth.org"
"1-Day Guide to Nusa Penida Island, Indonesia - Let's Travel it Up"
"Kelingkling beach from above | Etsy"
"Bali Travel Guide & Tips | Condé Nast Traveler"
"Bali Blue Dream Beach Background, Bali, Lembongan, Beach Background ..."
"Klumbu Indonesia"

"巴厘岛-梦开始的地方.在异国他乡遇见你所有的美好.-巴厘岛旅游攻略-游记-去哪儿攻略"
"【乌鲁瓦图情人崖旅游】乌鲁瓦图情人崖旅游攻略，乌鲁瓦图情人崖旅游景点大全-去哪儿网"
"巴厘岛-梦开始的地方.在异国他乡遇见你所有的美好.-巴厘岛旅游攻略-游记-去哪儿攻略"
"Perfect 3-day itinerary in Nusa Penida, Indonesia | Feast of Travel"
"Indonesia ~ Martina Move"
"7 Nights in Bali - Rewards4earth.org"
"1-Day Guide to Nusa Penida Island, Indonesia - Let's Travel it Up"
"Kelingkling beach from above | Etsy"
"B

# agent

In [28]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
# from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


# Define the tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


# tools = [search]

tool_node = ToolNode(tools)

# model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    # response = model.invoke(messages)
    response = model_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
# final_state = app.invoke(
#     {"messages": [HumanMessage(content="what is the weather in sf")]},
#     config={"configurable": {"thread_id": 42}}
# )

final_state = app.invoke(
    {"messages": [HumanMessage(content="计算下面两个数字之和：4654,135465")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'这两个数字的和是140,119。'

In [29]:
135465+4654

140119